In [1]:
# Setting up execution path
import os

print(f"Current working directory: {os.path.basename(os.getcwd())}")

# Change to root directory
os.chdir("../")
print(f"Current working directory (Changed): {os.path.basename(os.getcwd())}")

Current working directory: notebooks
Current working directory (Changed): Analyzing-Pokemons


In [2]:
from rich import print

In [3]:
from src.constants import CONFIGS
from src.utils.basic_utils import read_yaml

In [4]:
configs = read_yaml(CONFIGS).data_preprocessor
print(configs.to_dict())

[2024-02-24 05:01:24 PM]:ProjectLogger INFO:basic_utils 40 - yaml file: conf\configs.yaml loaded successfully


{'scraped_data_path': 'data/external/scraped_pokemons.csv', 'processed_data_path': 'data/processed/pokemons.csv'}

In [5]:
#imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

# module setup
%matplotlib inline
pd.options.display.precision = 5
warnings.filterwarnings("ignore")

In [24]:
# Get the CSV filepath from configs
scraped_data_path = configs.scraped_data_path

# Read the CSV data
pokemons_df_main = pd.read_csv(scraped_data_path, index_col=False)

# create a copy of it for working
df = pokemons_df_main.copy(deep=True)

# View a glimpse of the data
df.head()

,rank,name,types,total_power,hit_points,attack,defense,special_attack,special_defense,speed,icon_url,details_url,scrape_ts
0,1,Bulbasaur,"Grass, Poison",318,45,49,49,65,65,45,https://img.pokemondb.net/sprites/scarlet-viol...,https://pokemondb.net/pokedex/bulbasaur,2024-02-24 15:05:21
1,2,Ivysaur,"Grass, Poison",405,60,62,63,80,80,60,https://img.pokemondb.net/sprites/scarlet-viol...,https://pokemondb.net/pokedex/ivysaur,2024-02-24 15:05:21
2,3,Venusaur,"Grass, Poison",525,80,82,83,100,100,80,https://img.pokemondb.net/sprites/scarlet-viol...,https://pokemondb.net/pokedex/venusaur,2024-02-24 15:05:21
3,3,Mega Venusaur,"Grass, Poison",625,80,100,123,122,120,80,https://img.pokemondb.net/sprites/scarlet-viol...,https://pokemondb.net/pokedex/venusaur,2024-02-24 15:05:21
4,4,Charmander,Fire,309,39,52,43,60,50,65,https://img.pokemondb.net/sprites/scarlet-viol...,https://pokemondb.net/pokedex/charmander,2024-02-24 15:05:21


In [25]:
custom_index_col = pd.RangeIndex(start=1000, stop=1000 + len(df), step=1, name="PokeID")

df.index = custom_index_col
df.index = "P" + df.index.astype("string")
df = df.reset_index()

In [26]:
df.head()

,PokeID,rank,name,types,total_power,hit_points,attack,defense,special_attack,special_defense,speed,icon_url,details_url,scrape_ts
0,P1000,1,Bulbasaur,"Grass, Poison",318,45,49,49,65,65,45,https://img.pokemondb.net/sprites/scarlet-viol...,https://pokemondb.net/pokedex/bulbasaur,2024-02-24 15:05:21
1,P1001,2,Ivysaur,"Grass, Poison",405,60,62,63,80,80,60,https://img.pokemondb.net/sprites/scarlet-viol...,https://pokemondb.net/pokedex/ivysaur,2024-02-24 15:05:21
2,P1002,3,Venusaur,"Grass, Poison",525,80,82,83,100,100,80,https://img.pokemondb.net/sprites/scarlet-viol...,https://pokemondb.net/pokedex/venusaur,2024-02-24 15:05:21
3,P1003,3,Mega Venusaur,"Grass, Poison",625,80,100,123,122,120,80,https://img.pokemondb.net/sprites/scarlet-viol...,https://pokemondb.net/pokedex/venusaur,2024-02-24 15:05:21
4,P1004,4,Charmander,Fire,309,39,52,43,60,50,65,https://img.pokemondb.net/sprites/scarlet-viol...,https://pokemondb.net/pokedex/charmander,2024-02-24 15:05:21


In [30]:
df["types_2"] = df["types"].str.split(",")

In [34]:
df.head()

,PokeID,rank,name,types,total_power,hit_points,attack,defense,special_attack,special_defense,speed,icon_url,details_url,scrape_ts,types_2
0,P1000,1,Bulbasaur,"Grass, Poison",318,45,49,49,65,65,45,https://img.pokemondb.net/sprites/scarlet-viol...,https://pokemondb.net/pokedex/bulbasaur,2024-02-24 15:05:21,"[Grass, Poison]"
1,P1001,2,Ivysaur,"Grass, Poison",405,60,62,63,80,80,60,https://img.pokemondb.net/sprites/scarlet-viol...,https://pokemondb.net/pokedex/ivysaur,2024-02-24 15:05:21,"[Grass, Poison]"
2,P1002,3,Venusaur,"Grass, Poison",525,80,82,83,100,100,80,https://img.pokemondb.net/sprites/scarlet-viol...,https://pokemondb.net/pokedex/venusaur,2024-02-24 15:05:21,"[Grass, Poison]"
3,P1003,3,Mega Venusaur,"Grass, Poison",625,80,100,123,122,120,80,https://img.pokemondb.net/sprites/scarlet-viol...,https://pokemondb.net/pokedex/venusaur,2024-02-24 15:05:21,"[Grass, Poison]"
4,P1004,4,Charmander,Fire,309,39,52,43,60,50,65,https://img.pokemondb.net/sprites/scarlet-viol...,https://pokemondb.net/pokedex/charmander,2024-02-24 15:05:21,[Fire]


In [35]:
# Explode the 'types_2' column
df_exploded = df.explode("types_2")
df_exploded

,PokeID,rank,name,types,total_power,hit_points,attack,defense,special_attack,special_defense,speed,icon_url,details_url,scrape_ts,types_2
0,P1000,1,Bulbasaur,"Grass, Poison",318,45,49,49,65,65,45,https://img.pokemondb.net/sprites/scarlet-viol...,https://pokemondb.net/pokedex/bulbasaur,2024-02-24 15:05:21,Grass
0,P1000,1,Bulbasaur,"Grass, Poison",318,45,49,49,65,65,45,https://img.pokemondb.net/sprites/scarlet-viol...,https://pokemondb.net/pokedex/bulbasaur,2024-02-24 15:05:21,Poison
1,P1001,2,Ivysaur,"Grass, Poison",405,60,62,63,80,80,60,https://img.pokemondb.net/sprites/scarlet-viol...,https://pokemondb.net/pokedex/ivysaur,2024-02-24 15:05:21,Grass
1,P1001,2,Ivysaur,"Grass, Poison",405,60,62,63,80,80,60,https://img.pokemondb.net/sprites/scarlet-viol...,https://pokemondb.net/pokedex/ivysaur,2024-02-24 15:05:21,Poison
2,P1002,3,Venusaur,"Grass, Poison",525,80,82,83,100,100,80,https://img.pokemondb.net/sprites/scarlet-viol...,https://pokemondb.net/pokedex/venusaur,2024-02-24 15:05:21,Grass
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1211,P2211,1024,Normal Form,Normal,450,90,65,85,65,85,60,https://img.pokemondb.net/sprites/scarlet-viol...,https://pokemondb.net/pokedex/terapagos,2024-02-24 15:05:22,Normal
1212,P2212,1024,Terastal Form,Normal,600,95,95,110,105,110,85,https://img.pokemondb.net/sprites/scarlet-viol...,https://pokemondb.net/pokedex/terapagos,2024-02-24 15:05:22,Normal
1213,P2213,1024,Stellar Form,Normal,700,160,105,110,130,110,85,https://img.pokemondb.net/sprites/scarlet-viol...,https://pokemondb.net/pokedex/terapagos,2024-02-24 15:05:22,Normal
1214,P2214,1025,Pecharunt,"Poison, Ghost",600,88,88,160,88,88,88,https://img.pokemondb.net/sprites/scarlet-viol...,https://pokemondb.net/pokedex/pecharunt,2024-02-24 15:05:22,Poison


In [ ]:
dim_pokemon_cols = ["PokeID", "rank", "name", "icon_url", "details_url"]
fact_stats = ["poke_id", "type", "stat", "value"]

in stat we have "hit_points", "attach", "defense", "special_attack", "special_defense", "speed"